## 빅데이터 실습

### 데이터포털 OpenAPI 활용

#### 데이터 수집

##### OpenAPI
- 환광관광공사_관광빅데이터 수집
    - 기본URL : http://apis.data.go.kr/B551011/DataLabService/metcoRegnVisitrDDList (https 아님!)
        - __https로 요청하면 에러 발생__
    - 파라미터(?뒤) : 

        - serviceKey = 본인서비스키
        - numOfRows = 한페이지 표시할 결과수(10)
        - pageNo = 페이지번호(1)
        - MobileOS = IOS,AND,WIN,ETC(기타)
        - startYmd = 시작일자
        - endYmd = 끝일자
        - MobileApp = AppTest

#### 추가모듈
- XML을 딕셔너리로 한 번 바꿔주는 모듈
- xmltodict
```python
!pip install xmltodict

In [3]:
## 필요라이브러리 등록
import requests
from bs4 import BeautifulSoup
import datetime
import json
import xmltodict
import pandas as pd

In [4]:
# 웹으로 OpenAPI 요청하기 위한 기본함수
def getRequestUrl(url):
    try:
        res = requests.get(url)
        if res.status_code  ==200: # 웹 URL 요청 결과 OK
            html = res.text
            dict = xmltodict.parse(html,encoding='utf-8')
            return dict# 웹으로 전달 될 때에는 URL, utf-8로 인코딩, 돌려받을 때는 utf-8 디코딩
    except Exception as e:
        print(f'호출 예외 : {e} 시간 : {datetime.datetime.now()}')
        return None

In [5]:
# 파라미터를 변경하면서 OpenAPI을 요청할 URL을 만들어주는 함수
# 만약 시작 일자와 종료 일자가 다르면 파라미터를 2개 받아야한다.
def getTouristDataService(yyyymmdd):
    service_url = "http://apis.data.go.kr/B551011/DataLabService/metcoRegnVisitrDDList"
    parameters = r"?serviceKey=l2SER5UdFnVMTCfW%2F75Op1w1xj0ZVJyWjxZCA4s2qcMkJIjXonsQeorj8k35mmylOV0ywqc75RoXXtagwXuUnQ%3D%3D"
    parameters += f"&numOfRows=60"
    parameters += f"&pageNo=1"
    parameters += f"&MobileOS=ETC"
    parameters += f"&MobileApp=AppTest"
    parameters += f"&startYmd={yyyymmdd}"
    parameters += f"&endYmd={yyyymmdd}"
    url = service_url + parameters
    responseDecode = getRequestUrl(url)
    
    if responseDecode == None:
        return None
    else:
        content = responseDecode['response']['body']['items']['item']
        return content

In [6]:
getTouristDataService('20231202')

[{'areaCode': '11',
  'areaNm': '서울특별시',
  'daywkDivCd': '6',
  'daywkDivNm': '토요일',
  'touDivCd': '1',
  'touDivNm': '현지인(a)',
  'touNum': '4085018.0',
  'baseYmd': '20231202'},
 {'areaCode': '11',
  'areaNm': '서울특별시',
  'daywkDivCd': '6',
  'daywkDivNm': '토요일',
  'touDivCd': '2',
  'touDivNm': '외지인(b)',
  'touNum': '1940487.5',
  'baseYmd': '20231202'},
 {'areaCode': '11',
  'areaNm': '서울특별시',
  'daywkDivCd': '6',
  'daywkDivNm': '토요일',
  'touDivCd': '3',
  'touDivNm': '외국인(c)',
  'touNum': '31302.643199999977',
  'baseYmd': '20231202'},
 {'areaCode': '26',
  'areaNm': '부산광역시',
  'daywkDivCd': '6',
  'daywkDivNm': '토요일',
  'touDivCd': '1',
  'touDivNm': '현지인(a)',
  'touNum': '1479635.5',
  'baseYmd': '20231202'},
 {'areaCode': '26',
  'areaNm': '부산광역시',
  'daywkDivCd': '6',
  'daywkDivNm': '토요일',
  'touDivCd': '2',
  'touDivNm': '외지인(b)',
  'touNum': '484620.5',
  'baseYmd': '20231202'},
 {'areaCode': '26',
  'areaNm': '부산광역시',
  'daywkDivCd': '6',
  'daywkDivNm': '토요일',
  'touDivCd'

In [4]:
## 추가 라이브럴 ㅣ등록
from tqdm import tqdm
import time

In [ ]:
year = 2023
month = 12
touristDatas = []
for day in tqdm(range(1,32)):
    reqYmd = f'{year}{month}{day:02d}'
    if getTouristDataService(reqYmd) == None:
        resList = []
    else:
        resList = getTouristDataService(reqYmd)
    
    touristDatas = touristDatas + resList